In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJson, GeoJsonTooltip
import ipywidgets as widgets
from IPython.display import display, HTML

# ----------------------------------------
# 1) CSV laden
# ----------------------------------------
df = pd.read_csv("Kinos_legende.csv", sep=";")
df = df[df["Kantone"].notna() & (df["Kantone"] != "Total")].reset_index(drop=True)

years = [str(y) for y in range(2014, 2025)]
for y in years:
    df[y] = pd.to_numeric(df[y], errors="coerce")

# Mapping: deutsche CSV-Namen → GeoJSON-Namen
name_map = {
    "Zürich": "Zürich",
    "Bern": "Bern",
    "Luzern": "Luzern",
    "Uri": "Uri",
    "Schwyz": "Schwyz",
    "Obwalden": "Obwalden",
    "Nidwalden": "Nidwalden",
    "Glarus": "Glarus",
    "Zug": "Zug",
    "Freiburg": "Fribourg",
    "Solothurn": "Solothurn",
    "Basel-Stadt": "Basel-Stadt",
    "Basel-Landschaft": "Basel-Landschaft",
    "Schaffhausen": "Schaffhausen",
    "Appenzell A. Rh.": "Appenzell Ausserrhoden",
    "Appenzell I. Rh.": "Appenzell Innerrhoden",
    "St. Gallen": "St. Gallen",
    "Graubünden": "Graubünden",
    "Aargau": "Aargau",
    "Thurgau": "Thurgau",
    "Tessin": "Ticino",
    "Waadt": "Vaud",
    "Wallis": "Valais",
    "Neuenburg": "Neuchâtel",
    "Genf": "Genève",
    "Jura": "Jura",
}
df["kanton_geo"] = df["Kantone"].map(name_map)

# ----------------------------------------
# 2) GeoJSON laden
# ----------------------------------------
gdf = gpd.read_file("swiss-cantons-plotly.geojson")

# ----------------------------------------
# 3) Funktion zur Erstellung eines Karten-HTML
# ----------------------------------------
def make_map_html(year):
    df_year = df[["kanton_geo", year]].rename(columns={year: "Anzahl"})
    merged = gdf.merge(df_year, left_on="name", right_on="kanton_geo")

    m = folium.Map(location=[46.8, 8.3], zoom_start=7, tiles="cartodbpositron")

    folium.Choropleth(
        geo_data=merged.to_json(),
        data=merged,
        columns=["name", "Anzahl"],
        key_on="feature.properties.name",
        fill_color="YlOrRd",
        fill_opacity=0.8,
        line_opacity=0.8,
        line_color="black",
        legend_name=f"Anzahl Kinos ({year})",
    ).add_to(m)

    GeoJson(
        merged.to_json(),
        tooltip=GeoJsonTooltip(
            fields=["name", "Anzahl"],
            aliases=["Kanton:", "Kinos:"],
            sticky=True
        ),
        style_function=lambda x: {"fillOpacity": 0, "color": "transparent"},
    ).add_to(m)

    # 🎯 Karte in HTML konvertieren
    return m._repr_html_()

# ----------------------------------------
# 4) Widget-Container für die Karte
# ----------------------------------------
map_out = widgets.Output()

# ----------------------------------------
# 5) Update-Funktion
# ----------------------------------------
def update_map(change):
    with map_out:
        map_out.clear_output(wait=True)
        year = change["new"]
        html = make_map_html(year)
        display(HTML(html))

# ----------------------------------------
# 6) Dropdown erstellen
# ----------------------------------------
dropdown = widgets.Dropdown(
    options=years,
    value="2019",
    description="Jahr:",
)

dropdown.observe(update_map, names="value")

# ----------------------------------------
# 7) Alles anzeigen
# ----------------------------------------
display(dropdown)
update_map({"new": "2019"})   # initiale Karte laden
display(map_out)


Dropdown(description='Jahr:', index=5, options=('2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'…

Output()